In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import lxml
import json
import tqdm 
from PIL import Image
from urllib.request import urlopen,Request
from io import BytesIO
from datetime import datetime
import time
import os
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
ruta = os.getcwd()

In [5]:
ruta

'/Users/cevalenciam/Library/CloudStorage/OneDrive-Personal/SmartBuy/smartbuy/web-scraper-tottus'

In [4]:
def get_url_tottus(pais:str):
    
    if pais == 'CL':
        url_tottus = 'https://tottus.falabella.com/tottus-cl'
    elif pais == 'PE':
        url_tottus = 'https://tottus.falabella.com.pe/tottus-pe'
    else:
        return('País incorrecto')

    return(url_tottus)

In [5]:
def get_url(pais:str, url=''):

    headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0"}

    if url == '':
        url = get_url_tottus(pais)
    else:
        url = url

    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "lxml")
    all_data = soup.find_all("script", {"type": "application/json"})

    for data in all_data:
        jsn = json.loads(data.string)

    return jsn

In [6]:
data = get_url('CL')

In [7]:
with open(ruta + '/data.json', 'w') as file:
    json.dump(data, file, indent=4)

In [8]:
def get_info_category(pais:str):

    first_level_category = list()
    url_first_level_category = list()
    second_level_category = list()
    url_second_level_category = list()
    third_level_category = list()
    url_third_level_category = list()

    try:

        categories = get_url(pais)['props']['pageProps']['serverData']['headerData']['sisNavigationMenu']['entry']['categories']

        for i in categories:
            for j in i['second_level_categories']:
                for k in j['third_level_categories']:

                    first_level_category.append(i['item_name'])
                    url_first_level_category.append(i['item_url']['href'])
                    second_level_category.append(j['item_name'])
                    url_second_level_category.append(j['item_url'])
                    third_level_category.append(k['item_name'])
                    url_third_level_category.append(k['item_url'])

    except:

        df_url_cat = pd.DataFrame()

    df_url_cat = pd.DataFrame()

    df_url_cat['first_level_category'] = first_level_category
    df_url_cat['url_first_level_category'] = url_first_level_category
    df_url_cat['second_level_category'] = second_level_category
    df_url_cat['url_second_level_category'] = url_second_level_category
    df_url_cat['third_level_category'] = third_level_category
    df_url_cat['url_third_level_category'] = url_third_level_category

    return df_url_cat, len(third_level_category)

In [9]:
df_url_cat, num_cat = get_info_category('CL')

In [10]:
df_url_cat

,first_level_category,url_first_level_category,second_level_category,url_second_level_category,third_level_category,url_third_level_category
0,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Aceites,https://tottus.falabella.com/tottus-cl/categor...
1,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Pastas y Salsas,https://tottus.falabella.com/tottus-cl/categor...
2,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,"Arroz, Legumbres y Semillas",https://tottus.falabella.com/tottus-cl/categor...
3,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Conservas,https://tottus.falabella.com/tottus-cl/categor...
4,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Aderezos y Condimentos,https://tottus.falabella.com/tottus-cl/categor...
...,...,...,...,...,...,...
283,Hogar y Aire Libre,https://www.falabella.com/falabella-cl/categor...,Mundo Escolar,https://tottus.falabella.com/tottus-cl/categor...,Cuadernos,https://tottus.falabella.com/tottus-cl/categor...
284,Hogar y Aire Libre,https://www.falabella.com/falabella-cl/categor...,Mundo Escolar,https://tottus.falabella.com/tottus-cl/categor...,Útiles Escolares,https://tottus.falabella.com/tottus-cl/categor...
285,Hogar y Aire Libre,https://www.falabella.com/falabella-cl/categor...,Mundo Escolar,https://tottus.falabella.com/tottus-cl/categor...,Papeles,https://tottus.falabella.com/tottus-cl/categor...
286,Hogar y Aire Libre,https://www.falabella.com/falabella-cl/categor...,Mundo Escolar,https://tottus.falabella.com/tottus-cl/categor...,Mochilas Escolares,https://tottus.falabella.com/tottus-cl/categor...


In [11]:
def get_pages_categories(pais:str, df_categories:pd.DataFrame):

    total_url_cat = list()

    for i, first, second, third in tqdm.tqdm(zip(df_categories.url_third_level_category, df_categories.first_level_category, df_categories.second_level_category, df_categories.third_level_category)):

        total_categories = get_url(pais, url=i)

        try:
            per_page, count = total_categories['props']['pageProps']['pagination']['perPage'], total_categories['props']['pageProps']['pagination']['count']
            n_paginas = int(np.ceil(count/per_page))
            
            if n_paginas>1:
                url_cat_page = [[i + '?subdomain=tottus&page=' + str(j) + '&store=tottus', first, second, third] for j in range(2, n_paginas+1)]
                total_url_cat.append([i, first, second, third])
                total_url_cat.extend(url_cat_page)
            else:
                total_url_cat.append([i, first, second, third])
        
        except:
            total_url_cat.append([i, first, second, third])
    
    return(total_url_cat)

In [12]:
df_url_cat.shape

(288, 6)

In [13]:
list_pages_categories = get_pages_categories('CL', df_url_cat.iloc[:20,:])

20it [00:32,  1.61s/it]


In [14]:
list_pages_categories

[['https://tottus.falabella.com/tottus-cl/category/cat14090030/Aceites',
  'Despensa',
  'Abarrotes',
  'Aceites'],
 ['https://tottus.falabella.com/tottus-cl/category/CATG10210/Pastas-y-salsas',
  'Despensa',
  'Abarrotes',
  'Pastas y Salsas'],
 ['https://tottus.falabella.com/tottus-cl/category/CATG10207/Arroz--Legumbres-y-Semillas',
  'Despensa',
  'Abarrotes',
  'Arroz, Legumbres y Semillas'],
 ['https://tottus.falabella.com/tottus-cl/category/cat14090031/Conservas',
  'Despensa',
  'Abarrotes',
  'Conservas'],
 ['https://tottus.falabella.com/tottus-cl/category/CATG10213/Vinagres--aderezos-y-condimentos?subdomain=tottus',
  'Despensa',
  'Abarrotes',
  'Aderezos y Condimentos'],
 ['https://tottus.falabella.com/tottus-cl/category/CATG10194/Despensa',
  'Despensa',
  'Abarrotes',
  'Ver Todo'],
 ['https://tottus.falabella.com/tottus-cl/category/CATG10194/Despensa?subdomain=tottus&page=2&store=tottus',
  'Despensa',
  'Abarrotes',
  'Ver Todo'],
 ['https://tottus.falabella.com/tottus-c

In [15]:
def get_info_product(pais:str, df_info_category:list):

    url_category = list()
    name_product = list()
    url_product = list()
    first_level_category = list()
    second_level_category = list()
    third_level_category = list()

    for i in tqdm.tqdm(df_info_category):

        sku = get_url(pais, url=i[0])

        try:

            for j in sku['props']['pageProps']['results']:
                url_category.append(i[0])
                name_product.append(j['displayName'])
                url_product.append(j['url'])
                first_level_category.append(i[1])
                second_level_category.append(i[2])
                third_level_category.append(i[3])

        except:
            print('ERROR: url not found')
    
    df_url_sku = pd.DataFrame()

    df_url_sku['url_category'] = url_category
    df_url_sku['name_product'] = name_product
    df_url_sku['url_product'] = url_product
    df_url_sku['first_level_category'] = first_level_category
    df_url_sku['second_level_category'] = second_level_category
    df_url_sku['third_level_category'] = third_level_category

    return(df_url_sku.drop_duplicates())

In [16]:
df_url_sku = get_info_product('CL', list_pages_categories)

 87%|████████▋ | 20/23 [00:10<00:02,  1.35it/s]

ERROR: url not found


100%|██████████| 23/23 [00:11<00:00,  2.08it/s]


In [17]:
list_pages_categories[-4]

['https://tottus.falabella.com/tottus-cl/category/CATG10225/Leches-liquidas',
 'Despensa',
 'Lácteos y Frescos',
 'Leches Líquidas']

In [18]:
df_url_sku['url_category'].nunique()

22

In [19]:
def validate_url_cat(total_url_sku:pd.DataFrame, df_categories:pd.DataFrame):

    cat_validate = total_url_sku[['url_category']].drop_duplicates()
    cat_validate['url_down'] = 0

    cat_validate = df_categories.merge(cat_validate.rename(columns = {'url_category':'url_third_level_category'}), how='left')
    cat_validate['url_down'] = cat_validate.url_down.fillna(1)

    return cat_validate

In [20]:
validate_url_cat(df_url_sku, df_url_cat)

,first_level_category,url_first_level_category,second_level_category,url_second_level_category,third_level_category,url_third_level_category,url_down
0,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Aceites,https://tottus.falabella.com/tottus-cl/categor...,0.0
1,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Pastas y Salsas,https://tottus.falabella.com/tottus-cl/categor...,0.0
2,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,"Arroz, Legumbres y Semillas",https://tottus.falabella.com/tottus-cl/categor...,0.0
3,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Conservas,https://tottus.falabella.com/tottus-cl/categor...,0.0
4,Despensa,https://www.falabella.com/falabella-cl/categor...,Abarrotes,https://tottus.falabella.com/tottus-cl/categor...,Aderezos y Condimentos,https://tottus.falabella.com/tottus-cl/categor...,0.0
...,...,...,...,...,...,...,...
283,Hogar y Aire Libre,https://www.falabella.com/falabella-cl/categor...,Mundo Escolar,https://tottus.falabella.com/tottus-cl/categor...,Cuadernos,https://tottus.falabella.com/tottus-cl/categor...,1.0
284,Hogar y Aire Libre,https://www.falabella.com/falabella-cl/categor...,Mundo Escolar,https://tottus.falabella.com/tottus-cl/categor...,Útiles Escolares,https://tottus.falabella.com/tottus-cl/categor...,1.0
285,Hogar y Aire Libre,https://www.falabella.com/falabella-cl/categor...,Mundo Escolar,https://tottus.falabella.com/tottus-cl/categor...,Papeles,https://tottus.falabella.com/tottus-cl/categor...,1.0
286,Hogar y Aire Libre,https://www.falabella.com/falabella-cl/categor...,Mundo Escolar,https://tottus.falabella.com/tottus-cl/categor...,Mochilas Escolares,https://tottus.falabella.com/tottus-cl/categor...,1.0


In [21]:
data_sku = get_url('CL', df_url_sku.iloc[0, -4])

In [22]:
df_url_sku.iloc[0, -4]

'https://tottus.falabella.com/tottus-cl/product/110615848/aceite-maiz-1-lt-ch?exp=tottus'

In [23]:
with open(ruta + '/data_sku.json', 'w') as file:
    json.dump(data_sku, file, indent=4)

In [24]:
def get_info_image(df_info_sku:list):

    img_url = ''
    px_url = ''

    n_imagenes = len(df_info_sku)

    for i in range(n_imagenes):

        try:

            url_img = Request(df_info_sku[i]['url'],headers={'User-Agent': 'Mozilla/5.0'})

            u = urlopen(url_img)
            raw_data = u.read()
            u.close()

            img = Image.open(BytesIO(raw_data))

            if i == n_imagenes-1:
                img_url = img_url + df_info_sku[i]['url'] 
                px_url = px_url + str(img.size) 
            
            else:
                img_url = img_url + df_info_sku[i]['url'] + '; '
                px_url = px_url + str(img.size) + '; '

        except:
            
            if i == n_imagenes-1:

                img_url = img_url + 'url_image not found'
                px_url = px_url + ''

            else:

                img_url = img_url + 'url_image not found' + '; '
                px_url = px_url + '; '

    return img_url, px_url

In [25]:
def get_scraping_sku(pais_sku:str, url_sku:list):

    try:

        df_producto = get_url(pais_sku,url_sku[0])['props']['pageProps']['productData']

        try:
            id_url = df_producto['id']
        except:
            id_url = ''

        try: 
            name_url = df_producto['name']
        except:
            name_url = ''

        try:
            description_url =  df_producto['description']
        except:
            description_url = ''

        try:
            n_attributes = len(df_producto['attributes']['specifications'])
            atr_sku = ''

            for i in range(n_attributes):

                if i == n_attributes-1:
                    atr_sku = atr_sku+df_producto['attributes']['specifications'][i]['name']+': '+df_producto['attributes']['specifications'][i]['value']
                else:
                    atr_sku = atr_sku + df_producto['attributes']['specifications'][i]['name']+': '+df_producto['attributes']['specifications'][i]['value']+'; '

        except:
            atr_sku = ''
            
        try:

            normal_price = '' 
            internet_price = ''
            cmr_price = ''

            if len(df_producto['variants'][0]['prices'])>0:
            
                for j in df_producto['variants'][0]['prices']:

                    if j['type'] == 'normalPrice':
                        normal_price = j['price'][0]
                    elif j['type'] == 'internetPrice':
                        internet_price = j['price'][0]
                    elif j['type'] == 'cmrPrice':
                        cmr_price = j['price'][0]

        except:
                normal_price = ''
                internet_price = ''
                cmr_price = ''

        try:    
            id_client_sku = df_producto['variants'][0]['offerings'][0]['offeringId']
        except:
            id_client_sku = ''

        try:
            cod_sku = df_producto['variants'][0]['offerings'][0]['sellerSkuId']
        except:
            cod_sku = ''

        try:
            image_info_1,image_info_2 = get_info_image(df_producto['variants'][0]['medias'])
        except:
            image_info_1,image_info_2 = '',''

        try:
            first = url_sku[1]
        except:
            first = ''
        
        try:
            second = url_sku[2]
        except:
            second = ''
        
        try:
            third = url_sku[3]
        except:
            third = ''
        
        try:
            category_1 = df_producto['breadCrumb'][1]['label']
        except:
            category_1 = ''

        try:
            category_2 = df_producto['breadCrumb'][0]['label']
        except:
            category_2 = ''

    except:
        
        id_url = ''
        name_url = ''
        description_url = ''
        atr_sku = ''
        normal_price = ''
        internet_price = ''
        cmr_price = ''
        id_client_sku = ''
        cod_sku = ''
        image_info_1 = ''
        image_info_2 = ''
        first = ''
        second = ''
        third = ''
        category_1 = ''
        category_2 = ''

    return id_url, name_url, description_url, atr_sku, normal_price, internet_price, cmr_price, id_client_sku, cod_sku, image_info_1, image_info_2, first, second, third, category_1, category_2

In [36]:
lista = df_url_sku[['url_product', 'first_level_category', 'second_level_category', 'third_level_category']].values.tolist()
lista

[['https://tottus.falabella.com/tottus-cl/product/110615848/aceite-maiz-1-lt-ch?exp=tottus',
  'Despensa',
  'Abarrotes',
  'Aceites'],
 ['https://tottus.falabella.com/tottus-cl/product/110607052/tottus-aceite-de-oliva-tottus-500-ml-20525049?exp=tottus',
  'Despensa',
  'Abarrotes',
  'Aceites'],
 ['https://tottus.falabella.com/tottus-cl/product/110614785/salsa-toscana-al-pesto-200-grs?exp=tottus',
  'Despensa',
  'Abarrotes',
  'Pastas y Salsas'],
 ['https://tottus.falabella.com/tottus-cl/product/110610643/spaghetti-sin-gluten-vivo-250-gr?exp=tottus',
  'Despensa',
  'Abarrotes',
  'Pastas y Salsas'],
 ['https://tottus.falabella.com/tottus-cl/product/110611303/salsa-de-tomate-doy-pack-200-g-tottus?exp=tottus',
  'Despensa',
  'Abarrotes',
  'Pastas y Salsas'],
 ['https://tottus.falabella.com/tottus-cl/product/110616150/fideo-espiral-49--400-grs-111112-c?exp=tottus',
  'Despensa',
  'Abarrotes',
  'Pastas y Salsas'],
 ['https://tottus.falabella.com/tottus-cl/product/110615736/fideo-cor

In [27]:
lista[0]

['https://tottus.falabella.com/tottus-cl/product/110615848/aceite-maiz-1-lt-ch?exp=tottus',
 'Despensa',
 'Abarrotes',
 'Aceites']

In [28]:
get_scraping_sku('CL', lista[0])

('110615848',
 'ACEITE MAIZ 1 LT. CH',
 'ACEITE MAIZ 1 LT. CH',
 'Marca: chef; Tipo: Aceites Especiales; Formato: 1 LT; Unidades Por Paquete: 1; Energia Por 100 Gr Kcal: 828 kcal; Energia Por Porcion Kcal: 41 kcal; Proteinas Por 100 Gr G: 0 g; Proteinas Por Porcion G: 0 g; Grasa Total Por 100 Gr G: 92 g; Grasa Total Por Porcion G: 4.6 g; Hidratos De Carbono Disponibles Por 100 Gr G: 0 g; Hidratos De Carbono Disponibles Por Porcion G: 0 g; Azucares Totales Por 100 Gr G: 0 g; Azucares Totales Por Porcion G: 0 g; Sodio Por 100 Gr Mg: 0 mg; Sodio Por Porcion Mg: 0 mg; Condición del producto: Nuevo',
 '',
 '5.200',
 '',
 '110615849',
 '01020055',
 'https://s7d2.scene7.com/is/image/Tottus/01020055_1',
 '(400, 400)',
 'Despensa',
 'Abarrotes',
 'Aceites',
 'Alimentos y bebidas-Despensa',
 'Aceites')

In [29]:
df_producto = get_url('CL',lista[0][0])['props']['pageProps']['productData']

In [30]:
df_producto['breadCrumb'][1]['label']

'Alimentos y bebidas-Despensa'

In [31]:
df_producto['breadCrumb'][0]['label']

'Aceites'

In [32]:
def get_df_scraping(pais:str, list_url:list):

    pais_skus = list()
    url_skus = list()
    id = list()
    id_client = list()
    name = list()
    description = list()
    attributes = list()
    normal_prices = list()
    internet_prices = list()
    cmr_prices = list()
    cod_skus = list()
    url_images = list()
    px_images = list()
    first_category = list()
    second_category = list()
    third_category = list()
    category_1 = list()
    category_2 = list()

    for li in tqdm.tqdm(list_url):

        # print(sku)

        id_sku, name_sku, description_sku, atr_sku, normal_price, internet_price, cmr_price, id_client_sku, cod_sku, image_info_1, image_info_2, first, second, third, cat_1, cat_2 = get_scraping_sku(pais, li)

        pais_skus.append(pais)
        url_skus.append(li[0])
        id.append(id_sku)
        name.append(name_sku)
        description.append(description_sku)
        attributes.append(atr_sku)
        normal_prices.append(normal_price)
        internet_prices.append(internet_price)
        cmr_prices.append(cmr_price)
        id_client.append(id_client_sku)
        cod_skus.append(cod_sku)
        url_images.append(image_info_1)
        px_images.append(image_info_2)
        first_category.append(first)
        second_category.append(second)
        third_category.append(third)
        category_1.append(cat_1)
        category_2.append(cat_2)

    df_producto_atr = pd.DataFrame()

    df_producto_atr['pais'] = pais_skus
    df_producto_atr['url_sku'] = url_skus
    df_producto_atr['id'] = id
    df_producto_atr['name'] = name
    df_producto_atr['description'] = description
    df_producto_atr['id_client'] = id_client
    df_producto_atr['cod_sku'] = cod_skus
    df_producto_atr['first_category'] = first_category
    df_producto_atr['second_category'] = second_category
    df_producto_atr['third_category'] = third_category
    df_producto_atr['category_1'] = category_1
    df_producto_atr['category_2'] = category_2

    df_producto_atr['attributes'] = attributes

    df_producto_atr['normal_price'] = normal_prices
    df_producto_atr['internet_price'] = internet_prices
    df_producto_atr['cmr_price'] = cmr_prices

    df_producto_atr['url_image'] = url_images
    df_producto_atr['pixel_image'] = px_images

    return(df_producto_atr)

In [33]:
lista[:5]

[['https://tottus.falabella.com/tottus-cl/product/110615848/aceite-maiz-1-lt-ch?exp=tottus',
  'Despensa',
  'Abarrotes',
  'Aceites'],
 ['https://tottus.falabella.com/tottus-cl/product/110607052/tottus-aceite-de-oliva-tottus-500-ml-20525049?exp=tottus',
  'Despensa',
  'Abarrotes',
  'Aceites'],
 ['https://tottus.falabella.com/tottus-cl/product/110614785/salsa-toscana-al-pesto-200-grs?exp=tottus',
  'Despensa',
  'Abarrotes',
  'Pastas y Salsas'],
 ['https://tottus.falabella.com/tottus-cl/product/110610643/spaghetti-sin-gluten-vivo-250-gr?exp=tottus',
  'Despensa',
  'Abarrotes',
  'Pastas y Salsas'],
 ['https://tottus.falabella.com/tottus-cl/product/110611303/salsa-de-tomate-doy-pack-200-g-tottus?exp=tottus',
  'Despensa',
  'Abarrotes',
  'Pastas y Salsas']]

In [34]:
df_final = get_df_scraping('CL', lista[:5])

100%|██████████| 5/5 [00:08<00:00,  1.78s/it]


In [35]:
df_final.head()

,pais,url_sku,id,name,description,id_client,cod_sku,first_category,second_category,third_category,category_1,category_2,attributes,normal_price,internet_price,cmr_price,url_image,pixel_image
0,CL,https://tottus.falabella.com/tottus-cl/product...,110615848,ACEITE MAIZ 1 LT. CH,ACEITE MAIZ 1 LT. CH,110615849,01020055,Despensa,Abarrotes,Aceites,Alimentos y bebidas-Despensa,Aceites,Marca: chef; Tipo: Aceites Especiales; Formato...,,5.200,,https://s7d2.scene7.com/is/image/Tottus/010200...,"(400, 400)"
1,CL,https://tottus.falabella.com/tottus-cl/product...,110607052,Aceite de Oliva Extra Virgen - 500 ML,"<link href=""https://static.okto.shop/css/oktos...",110607053,20525049,Despensa,Abarrotes,Aceites,Alimentos y bebidas-Despensa,Aceites,Marca: tottus; Tipo: Aceites de Oliva; Formato...,,5.490,,https://s7d2.scene7.com/is/image/Tottus/205250...,"(400, 400)"
2,CL,https://tottus.falabella.com/tottus-cl/product...,110614785,SALSA TOSCANA AL PESTO 200 GRS..,SALSA TOSCANA AL PESTO 200 GRS..,110614786,20013926,Despensa,Abarrotes,Pastas y Salsas,Pastas y salsas,Salsas,Formato: 200 GR,,940,,https://s7d2.scene7.com/is/image/Tottus/200139...,"(400, 400)"
3,CL,https://tottus.falabella.com/tottus-cl/product...,110610643,SPAGHETTI SIN GLUTEN VIVO 250 GR,SPAGHETTI SIN GLUTEN VIVO 250 GR,110610646,20351850,Despensa,Abarrotes,Pastas y Salsas,Pastas y salsas,Pastas,Marca: vivo; Tipo: Pastas Sin Gluten; Formato:...,,2.450,,https://s7d2.scene7.com/is/image/Tottus/203518...,"(400, 400)"
4,CL,https://tottus.falabella.com/tottus-cl/product...,110611303,SALSA DE TOMATE DOY PACK 200 G. TOTTUS.,SALSA DE TOMATE DOY PACK 200 G. TOTTUS.,110611304,20135745,Despensa,Abarrotes,Pastas y Salsas,Pastas y salsas,Salsas,Marca: tottus; Tipo: Salsa de Tomate; Formato:...,,690,,https://s7d2.scene7.com/is/image/Tottus/201357...,"(400, 400)"
